In [1]:
!pip install tiingo
!pip install tiingo[pandas]

In [30]:
# Importing necessary libraries 
import os
import json
import pandas as pd
import numpy as np
import glob
from IPython import get_ipython


# For data cleaning and visualization
import pandas as pd
import numpy as np
from numpy import array
from datetime import date
from datetime import datetime, timedelta
import pandas_datareader.data as web


# For saving the model
import pickle

# For model Evaluation
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import math

import re
import nltk
from nltk.corpus import stopwords

import nltk
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [31]:
df = pd.read_csv("FinalData/news_headlines.csv", index_col=0)
df

,headline,date
0,Nikkei down 1 4 pct on economy fears autos drop,2008-10-02
1,FOREX Dollar poised for biggest weekly gain in...,2008-10-03
2,GLOBAL MARKETS Stocks rally on Wells Wachovia ...,2008-10-03
3,GLOBAL MARKETS Stocks hold up after Wells Wach...,2008-10-03
4,GLOBAL MARKETS U S bailout plan passes inves...,2008-10-03
...,...,...
221508,Why T Mobile s deal with Sprint could be the w...,2020-02-13
221509,6 Low Price to Book Stocks With Great Growth P...,2020-02-13
221510,Cisco Systems CSCO Q2 Earnings And Revenues ...,2020-02-13
221511,Improve Your Retirement Income With These 3 To...,2020-02-13


In [32]:
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

In [33]:
# Iterate through the headlines and get the polarity scores using vader
scores = df['headline'].apply(vader.polarity_scores).tolist()
scores

[{'neg': 0.45, 'neu': 0.55, 'pos': 0.0, 'compound': -0.5994},
 {'neg': 0.0, 'neu': 0.597, 'pos': 0.403, 'compound': 0.6597},
 {'neg': 0.0, 'neu': 0.645, 'pos': 0.355, 'compound': 0.5267},
 {'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound': 0.25},
 {'neg': 0.136, 'neu': 0.583, 'pos': 0.282, 'compound': 0.3612},
 {'neg': 0.463, 'neu': 0.537, 'pos': 0.0, 'compound': -0.7845},
 {'neg': 0.421, 'neu': 0.305, 'pos': 0.274, 'compound': -0.5267},
 {'neg': 0.339, 'neu': 0.661, 'pos': 0.0, 'compound': -0.6249},
 {'neg': 0.216, 'neu': 0.784, 'pos': 0.0, 'compound': -0.296},
 {'neg': 0.0, 'neu': 0.827, 'pos': 0.173, 'compound': 0.2235},
 {'neg': 0.0, 'neu': 0.606, 'pos': 0.394, 'compound': 0.6369},
 {'neg': 0.323, 'neu': 0.677, 'pos': 0.0, 'compound': -0.5106},
 {'neg': 0.538, 'neu': 0.462, 'pos': 0.0, 'compound': -0.8519},
 {'neg': 0.198, 'neu': 0.517, 'pos': 0.284, 'compound': 0.25},
 {'neg': 0.429, 'neu': 0.571, 'pos': 0.0, 'compound': -0.7184},
 {'neg': 0.462, 'neu': 0.538, 'pos': 0.0, 'compound'

In [34]:
# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)
scores_df

,neg,neu,pos,compound
0,0.450,0.550,0.000,-0.5994
1,0.000,0.597,0.403,0.6597
2,0.000,0.645,0.355,0.5267
3,0.000,0.800,0.200,0.2500
4,0.136,0.583,0.282,0.3612
...,...,...,...,...
221508,0.000,0.881,0.119,0.2263
221509,0.131,0.313,0.556,0.7783
221510,0.000,1.000,0.000,0.0000
221511,0.000,0.701,0.299,0.5719


In [35]:
# Join the DataFrames of the news and the list of dicts
scores_df = df.join(scores_df, rsuffix='_right')
scores_df

,headline,date,neg,neu,pos,compound
0,Nikkei down 1 4 pct on economy fears autos drop,2008-10-02,0.450,0.550,0.000,-0.5994
1,FOREX Dollar poised for biggest weekly gain in...,2008-10-03,0.000,0.597,0.403,0.6597
2,GLOBAL MARKETS Stocks rally on Wells Wachovia ...,2008-10-03,0.000,0.645,0.355,0.5267
3,GLOBAL MARKETS Stocks hold up after Wells Wach...,2008-10-03,0.000,0.800,0.200,0.2500
4,GLOBAL MARKETS U S bailout plan passes inves...,2008-10-03,0.136,0.583,0.282,0.3612
...,...,...,...,...,...,...
221508,Why T Mobile s deal with Sprint could be the w...,2020-02-13,0.000,0.881,0.119,0.2263
221509,6 Low Price to Book Stocks With Great Growth P...,2020-02-13,0.131,0.313,0.556,0.7783
221510,Cisco Systems CSCO Q2 Earnings And Revenues ...,2020-02-13,0.000,1.000,0.000,0.0000
221511,Improve Your Retirement Income With These 3 To...,2020-02-13,0.000,0.701,0.299,0.5719


In [20]:
scores_df[scores_df.isnull().any(axis=1)]

,headline,date,neg,neu,pos,compound


In [21]:
scores_df = scores_df.sort_values(by=['date'], ascending=True)
scores_df

,headline,date,neg,neu,pos,compound
0,Nikkei down 1 4 pct on economy fears autos drop,2008-10-02,0.450,0.550,0.000,-0.5994
1,FOREX Dollar poised for biggest weekly gain in...,2008-10-03,0.000,0.597,0.403,0.6597
2,GLOBAL MARKETS Stocks rally on Wells Wachovia ...,2008-10-03,0.000,0.645,0.355,0.5267
3,GLOBAL MARKETS Stocks hold up after Wells Wach...,2008-10-03,0.000,0.800,0.200,0.2500
4,GLOBAL MARKETS U S bailout plan passes inves...,2008-10-03,0.136,0.583,0.282,0.3612
...,...,...,...,...,...,...
221487,Comerica taps Wells Fargo exec to head wealth ...,2020-02-13,0.000,0.606,0.394,0.6369
221488,Top 5 Things to Know in the Market on Wednesda...,2020-02-13,0.000,0.847,0.153,0.2023
221489,Stocks making the biggest moves after hours C...,2020-02-13,0.000,1.000,0.000,0.0000
221480,Stocks U S Futures Point Higher as Virus Fe...,2020-02-13,0.248,0.531,0.221,-0.0772


In [39]:
mean_scores = scores_df.drop(columns = ['neg','neu','pos'])
mean_scores

,headline,date,compound
0,Nikkei down 1 4 pct on economy fears autos drop,2008-10-02,-0.5994
1,FOREX Dollar poised for biggest weekly gain in...,2008-10-03,0.6597
2,GLOBAL MARKETS Stocks rally on Wells Wachovia ...,2008-10-03,0.5267
3,GLOBAL MARKETS Stocks hold up after Wells Wach...,2008-10-03,0.2500
4,GLOBAL MARKETS U S bailout plan passes inves...,2008-10-03,0.3612
...,...,...,...
221508,Why T Mobile s deal with Sprint could be the w...,2020-02-13,0.2263
221509,6 Low Price to Book Stocks With Great Growth P...,2020-02-13,0.7783
221510,Cisco Systems CSCO Q2 Earnings And Revenues ...,2020-02-13,0.0000
221511,Improve Your Retirement Income With These 3 To...,2020-02-13,0.5719


In [40]:
mean_scores = mean_scores.groupby(['date']).agg(['mean'])
mean_scores

,compound
,mean
date,
2008-10-02,-0.599400
2008-10-03,0.449400
2008-10-06,-0.655600
2008-10-07,-0.460450
2008-10-08,0.223500
...,...
2020-02-09,0.074910
2020-02-10,0.097086


In [41]:
mean_scores['date'] = mean_scores.index
mean_scores = mean_scores.reset_index(drop=True)
mean_scores

,compound,date
,mean,
0,-0.599400,2008-10-02
1,0.449400,2008-10-03
2,-0.655600,2008-10-06
3,-0.460450,2008-10-07
4,0.223500,2008-10-08
...,...,...
3636,0.074910,2020-02-09
3637,0.097086,2020-02-10
3638,0.005268,2020-02-11


In [44]:
mean_scores['date'] = pd.to_datetime(mean_scores['date'])
mean_scores = mean_scores.reset_index(drop=True)
mean_scores

,compound,date
,mean,
0,-0.599400,2008-10-02
1,0.449400,2008-10-03
2,-0.655600,2008-10-06
3,-0.460450,2008-10-07
4,0.223500,2008-10-08
...,...,...
3636,0.074910,2020-02-09
3637,0.097086,2020-02-10
3638,0.005268,2020-02-11


In [45]:
mean_scores2 = pd.DataFrame(columns = {'date','mean_compound'})
mean_scores2['date'] = mean_scores['date']
mean_scores2['mean_compound'] = mean_scores['compound']['mean']
mean_scores2

,mean_compound,date
0,-0.599400,2008-10-02
1,0.449400,2008-10-03
2,-0.655600,2008-10-06
3,-0.460450,2008-10-07
4,0.223500,2008-10-08
...,...,...
3636,0.074910,2020-02-09
3637,0.097086,2020-02-10
3638,0.005268,2020-02-11
3639,0.041805,2020-02-12


In [46]:
scores_df.to_csv('FinalData/vadar_scores.csv')

In [47]:
mean_scores2.to_csv('FinalData/vadar_mean_compound_scores.csv')